<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/no-BN-and-dropout/cnnTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from google.colab import files

uploaded = files.upload()

Saving Model.py to Model (1).py


In [0]:
from Model import Model

In [0]:
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install -q torchsummary
    from torchsummary import summary


In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [0]:
cuda = True
train_batch_size = 256
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [0]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

In [0]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [0]:
def test(epoch, best_loss, best_epoch, lr):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best_"+str(lr)+".pt")
        try:
          if gsync_save:
              gsync.update_file_to_folder("best_"+str(lr)+".pt")
        except:
          print('Failed to gsync_save.')

        try:
          model_save_name = 'best_'+str(lr)
          path = F"/content/drive/My Drive/dl-reproducibility-project/model/{model_save_name}" 
          torch.save(model.state_dict(), path+'.epoch-{}.pt'.format(epoch))
        except:
          print('Failed to save best model to personal google drive')

    return best_loss, best_epoch, correct

In [0]:
try:
  os.makedirs('/content/drive/My Drive/dl-reproducibility-project/model/')
except:
  print('')

In [13]:
for lr in [0.1,0.05,0.01]:
  print("Training with ", lr)

  
  try: 
    del model
  except:
    print('')
  
  # Maybe it's redundant 
  try:
    del criterion
    del optimizer
    del scheduler
  except:
    print('')
  
  try:
    torch.cuda.empty_cache()
  except:
    print('')
  
  #
  # Chnage Here
  #
  # Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
  # Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
  # The scecond one is the strided. The third: ConvPool. The last: All CNN.
  model = Model(dropOut=False, BN=False)
  if cuda:
      model.cuda()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
  scheduler = optim.lr_scheduler.MultiStepLR(
      optimizer, milestones=[200, 250, 300], gamma=0.1)
  print(model)
  summary(model.model, input_size=(3, 32, 32))
  for epoch in range(350):
      scheduler.step()
      train(epoch)
      best_loss, best_epoch, acc = test(epoch, best_loss, best_epoch, lr)
      if (epoch>100)&(acc.item()>0)&((acc.item()/len(test_loader.dataset))<=0.1):
        print("Stop at acc of test set:",acc.item()/len(test_loader.dataset))
        break
  # if in Google Colab, download your model with this
  #from google.colab import files
  #files.download("best_"+str(lr)+"pt")

Training with  0.1


Model(
  (model): Sequential(
    (0): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (9): ReLU()
    (10): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (11): ReLU()
    (12): AdaptiveAvgPool2d(output_size=1)
    (13): Flatten()
  )
  (conv_3_192_192_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_192_192): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  (conv_1_192_class): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
  (BN_96): BatchNorm2d(96, eps=1e-05, momentum=0.

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.302589
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.162351

Test set: Average loss: 0.0144, Accuracy: 3673/10000 (37%)

Failed to gsync_save.
Train Epoch: 1 [0/50000 (0%)]	Loss: 1.761574
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.454118

Test set: Average loss: 0.0113, Accuracy: 4988/10000 (50%)



Uploading file best_0.1.pt: 100%|██████████| 100/100 [00:01<00:00, 70.05it/s]


Train Epoch: 2 [0/50000 (0%)]	Loss: 1.408949
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.215692

Test set: Average loss: 0.0097, Accuracy: 5943/10000 (59%)



...

Train Epoch: 37 [0/50000 (0%)]	Loss: 0.378875
Train Epoch: 37 [25600/50000 (51%)]	Loss: 0.466589

Test set: Average loss: 0.0045, Accuracy: 8221/10000 (82%)

Train Epoch: 38 [0/50000 (0%)]	Loss: 0.353376
Train Epoch: 38 [25600/50000 (51%)]	Loss: 0.307732

Test set: Average loss: 0.0050, Accuracy: 8056/10000 (81%)

Train Epoch: 39 [0/50000 (0%)]	Loss: 0.433022
Train Epoch: 39 [25600/50000 (51%)]	Loss: 0.431996

Test set: Average loss: 0.0046, Accuracy: 8176/10000 (82%)

Train Epoch: 40 [0/50000 (0%)]	Loss: 0.350525
Train Epoch: 40 [25600/50000 (51%)]	Loss: 0.473271

Test set: Average loss: 0.0052, Accuracy: 8084/10000 (81%)

Train Epoch: 41 [0/50000 (0%)]	Loss: 0.451470
Train Epoch: 41 [25600/50000 (51%)]	Loss: 0.366720

Test set: Average loss: 0.0047, Accuracy: 8219/10000 (82%)

Train Epoch: 42 [0/50000 (0%)]	Loss: 0.384213
Train Epoch: 42 [25600/50000 (51%)]	Loss: 0.357073

Test set: Average loss: 0.0049, Accuracy: 8123/10000 (81%)

Train Epoch: 43 [0/50000 (0%)]	Loss: 0.294119
Train 

Uploading file best_0.01.pt: 100%|██████████| 100/100 [00:01<00:00, 81.10it/s]


Train Epoch: 44 [0/50000 (0%)]	Loss: 0.339211
Train Epoch: 44 [25600/50000 (51%)]	Loss: 0.335791

Test set: Average loss: 0.0049, Accuracy: 8190/10000 (82%)

Train Epoch: 45 [0/50000 (0%)]	Loss: 0.300637
Train Epoch: 45 [25600/50000 (51%)]	Loss: 0.365958

Test set: Average loss: 0.0047, Accuracy: 8219/10000 (82%)

Train Epoch: 46 [0/50000 (0%)]	Loss: 0.334885
Train Epoch: 46 [25600/50000 (51%)]	Loss: 0.350493

Test set: Average loss: 0.0042, Accuracy: 8374/10000 (84%)



Uploading file best_0.01.pt: 100%|██████████| 100/100 [00:00<00:00, 112.99it/s]


Train Epoch: 47 [0/50000 (0%)]	Loss: 0.315269
Train Epoch: 47 [25600/50000 (51%)]	Loss: 0.365177

Test set: Average loss: 0.0044, Accuracy: 8361/10000 (84%)

Train Epoch: 48 [0/50000 (0%)]	Loss: 0.251937
Train Epoch: 48 [25600/50000 (51%)]	Loss: 0.290420

Test set: Average loss: 0.0047, Accuracy: 8245/10000 (82%)

Train Epoch: 49 [0/50000 (0%)]	Loss: 0.315251
Train Epoch: 49 [25600/50000 (51%)]	Loss: 0.357175

Test set: Average loss: 0.0045, Accuracy: 8319/10000 (83%)

Train Epoch: 50 [0/50000 (0%)]	Loss: 0.336443
Train Epoch: 50 [25600/50000 (51%)]	Loss: 0.262055

Test set: Average loss: 0.0044, Accuracy: 8372/10000 (84%)

Train Epoch: 51 [0/50000 (0%)]	Loss: 0.347130
Train Epoch: 51 [25600/50000 (51%)]	Loss: 0.333630

Test set: Average loss: 0.0046, Accuracy: 8277/10000 (83%)

Train Epoch: 52 [0/50000 (0%)]	Loss: 0.306624
Train Epoch: 52 [25600/50000 (51%)]	Loss: 0.299285

Test set: Average loss: 0.0043, Accuracy: 8378/10000 (84%)

Train Epoch: 53 [0/50000 (0%)]	Loss: 0.262817
Train 